# Week 1. Homework 1. 

# Libraries

In [2]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

# Data

Use the following commands to download the data locally:
 - `/workspaces/mlops-zoomcamp-2025/01-intro/data/`

 - `wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet` - get data for January 2023
  
 -  `wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet` - get data for February 2023

# Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [3]:
%%time

# Check data for January 2023
path_to_data = '../data/yellow_tripdata_2023-01.parquet'
df = pd.read_parquet(path_to_data)

CPU times: user 1.01 s, sys: 219 ms, total: 1.23 s
Wall time: 834 ms


In [4]:
df.shape

(3066766, 19)

In [5]:
print(f'Answer. Number of columns: {df.shape[1]}')

Answer. Number of columns: 19


# Question 2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [7]:
%%time 

# Calculate the duration of each trip in minutes
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)

df.head(2)

CPU times: user 6.44 s, sys: 641 ms, total: 7.08 s
Wall time: 7.13 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0,6.316667


In [8]:
print(f'Answer. Standard deviation of trip duration: {df["duration"].std():.2f} minutes')

Answer. Standard deviation of trip duration: 42.59 minutes


# Question 3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [9]:
trip_duration_1_to_60_condition = (df['duration'] >= 1) & (df['duration'] <= 60)

(trip_duration_1_to_60_condition).mean()

np.float64(0.9812202822125979)

In [10]:
print(f'Answer. Fraction of the records left after you dropped the outliers: {(trip_duration_1_to_60_condition).mean():.2%}')

Answer. Fraction of the records left after you dropped the outliers: 98.12%


In [11]:
# Drop outliers
df = df.loc[trip_duration_1_to_60_condition]

# Question 4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it
  
What's the dimensionality of this matrix (number of columns)?

In [12]:
# Re-cast id's into strings
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

In [13]:
%%time 

# Turn the dataframe with categorical features into a list of dictionaries
train_dicts = df[categorical].to_dict(orient='records')

CPU times: user 6.08 s, sys: 231 ms, total: 6.31 s
Wall time: 6.38 s


In [14]:
# Check the first 10 dictionaries
train_dicts[:10]

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'}]

In [15]:
%%time

# Fit a DictVectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

CPU times: user 4.29 s, sys: 136 ms, total: 4.43 s
Wall time: 4.46 s


In [16]:
X_train.shape

(3009173, 515)

In [17]:
print(f'Answer. Dimensionality of this matrix (number of columns): {X_train.shape[1]}')

Answer. Dimensionality of this matrix (number of columns): 515


# Question 5. Training a model

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters, where duration is the response variable
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [18]:
# Target variable

target = 'duration'
y_train = df[target].values

In [19]:
%%time 

lr = LinearRegression()
lr.fit(X_train, y_train)

CPU times: user 43.2 s, sys: 46.8 ms, total: 43.2 s
Wall time: 23.7 s


LinearRegression()

In [20]:
# Make predictions
y_pred = lr.predict(X_train)

In [21]:
# Evaluate the performance of the model: use RMSE 
root_mean_squared_error(y_train, y_pred)

# On average, the model is off by 7.65 minutes

7.6492624397080675

In [22]:
print(f'Answer. RMSE on train: {root_mean_squared_error(y_train, y_pred):.2f} minutes')

Answer. RMSE on train: 7.65 minutes


# Question 6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [23]:
df_val = pd.read_parquet('../data/yellow_tripdata_2023-02.parquet')

In [24]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2913955 entries, 0 to 2913954
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [25]:
df_val['duration'] = df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']

In [1]:
df_val['duration'] = df_val['duration'].apply(lambda td: td.total_seconds() / 60)

NameError: name 'df_val' is not defined

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

df_val[categorical] = df_val[categorical].astype(str)
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df_val[target].values
y_pred = lr.predict(X_val)